# Split null basis computation
By exploiting the structure of the $A$ matrix, it is possible to compute a matrix $B_1$ that may be reused for every line in the network. For each individual line, $n_t$ additional rows $B_2$ are computed so that $[B_1;B_2]$ forms a null basis for $A$. Not only does reusing $B_1$ save time, but $B_2$ is computed using only a sparse matrix multiplication (a cheap operation). Switching to this new null basis approach yields a speedup of forty percent for the algorithm overall. The remainder of this report details the new approach.

Most of the $A$ matrix is invariant with respect to which line the temporal instanton QCQP is focusing on. I refer to the fixed but network-dependent part as $A_1$, and the remaining part as $A_2$. Only $A_2$ changes from one line to the next. The $A$ matrix overall takes the following form:

$$
\begin{align}
A &= \begin{bmatrix} A_1 & 0 \\ A_2 & I \end{bmatrix},
\end{align}
$$
where $A_1$ contains the admittance matrix, participation factors, and decision variable coefficients, and $A_2$ defines auxiliary angle variables in terms of voltage angle differences. When moving from one line to the next, $A_2$ is updated to connect auxiliary angle variables to the proper nodal voltage angles. All other elements of $A$ are unaltered.

A null basis for $A$ is any matrix $B$ that satisfies the following condition:

$$
\begin{align}
AB &= 0 \\
\iff \begin{bmatrix}  A_1 & 0 \\ A_2 & I \end{bmatrix} \begin{bmatrix} B_1 \\ B_2 \end{bmatrix} &= 0 \\
\iff \begin{cases} A_1 B_1 &= 0 \\ B_2 &= -A_2 B_1 \end{cases}
\end{align}
$$

Observe that $B_1$ is a function of $A_1$ only. Because $A_1$ does not depend on the chosen line, $B_1$ may be computed outside the line loop. For an individual line, $B_2$ may be computed by multiplying $A_2$ and $B_1$. Stacking $B_1$ atop $B_2$ yields a matrix $B$ such that $AB = 0$, as desired.

The computational cost of this approach is one $(n_b+n_r+1)n_t$-by-$(n_b+1)n_t$ sparse QR decomposition plus $n_l$ (number of lines) sparse matrix multiplications, each with greatest dimension $(n_b + n_r + 1)n_t$. The figure below illustrates the effect of switching from the previous approach (performing a sparse QR decomposition for each line) to the new approach:

![](../images/2015-12-07-timing-analysis-reuse-temp.png)
![](../images/2016-01-19-timing-analysis-split-null-basis.png)

The speedup is roughly forty percent.

Note that the "after" figure does not account for the time taken to compute $B_1$, since that step is now performed outside the line loop.